# Svami Gap Analysis Demo

Call the /gap-analysis endpoint exposed by the Svami orchestrator to surface BigQuery-backed gap metrics directly inside a Kaggle notebook.


## Prerequisites
- Svami orchestrator service is running and reachable from this notebook.
- A valid API token (or disable auth for local testing).
- The 
equests library is available (pre-installed on Kaggle).
- The Kaggle runtime already includes `pandas` and IPython display utilities for inline rendering.


In [ ]:
import os
import requests
import pandas as pd
from IPython.display import Markdown, display

SVAMI_URL = os.getenv('SVAMI_URL', 'http://localhost:8003')
API_TOKEN = os.getenv('SVAMI_API_TOKEN', 'demo-token')

session = requests.Session()
session.headers.update({'Authorization': f'Bearer {API_TOKEN}'})

print(f'Using Svami at {SVAMI_URL}')


In [ ]:
payload = {
    'topic': 'FastAPI authentication',
    'artifact_type': 'fastapi',
    'limit': 5
}

response = session.post(f"{SVAMI_URL}/gap-analysis", json=payload, timeout=30)
response.raise_for_status()
analysis = response.json()

display(Markdown(f"### Gap Analysis for **{analysis['topic']}**"))
display(Markdown(analysis['message']))


In [ ]:
findings = analysis.get('findings', [])

if not findings:
    display(Markdown('_No gaps returned for this topic._'))
else:
    df = pd.DataFrame(findings)
    note = "_showing {results} findings (limit {limit})_".format(
        results=len(df), limit=payload['limit']
    )
    display(Markdown(note))
    display(df[['rule_name', 'artifact_type', 'severity', 'confidence', 'source_path', 'suggested_fix']])
